In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
from collections import deque
import matplotlib.pyplot as plt

class KPIOutlierMonitorJson:
    def __init__(self, config):
        self.config = config
        self.windows = {
            kpi: deque(maxlen=details['outlier_duration']) for kpi, details in config.items()
        }
        self.kpi_columns = list(config.keys())
        self.kpi_history = {
            kpi: [] for kpi in self.kpi_columns
        }

    def process_record(self, record):
        timestamp = record['datetime']
        alarms = []

        for kpi_name in self.kpi_columns:
            kpi_value = record.get(kpi_name, None)
            if kpi_value is None:
                raise ValueError(f"KPI '{kpi_name}' not found in the record.")

            
            self.kpi_history[kpi_name].append((timestamp, kpi_value))

            
            min_threshold = self.config[kpi_name]['min']
            max_threshold = self.config[kpi_name]['max']
            outlier_duration = self.config[kpi_name]['outlier_duration']

            
            if kpi_value > max_threshold or kpi_value < min_threshold:
                self.windows[kpi_name].append(True)
            else:
                self.windows[kpi_name].append(False)

            
            if len(self.windows[kpi_name]) == outlier_duration and all(self.windows[kpi_name]):
                alarms.append(f"Alarm triggered for {kpi_name} at {timestamp}")
                self.visualize_kpi(kpi_name, timestamp, kpi_value)

        return alarms

    def visualize_kpi(self, kpi_name, alarm_time, current_value):
        
        timestamps, values = zip(*self.kpi_history[kpi_name]) if self.kpi_history[kpi_name] else ([], [])

        
        min_threshold = self.config[kpi_name]['min']
        max_threshold = self.config[kpi_name]['max']

        
        mean_value = sum(value[1] for value in self.kpi_history[kpi_name]) / len(self.kpi_history[kpi_name]) if self.kpi_history[kpi_name] else 0

        
        if len(values) > 1:
            percentage_change = ((current_value - mean_value) / mean_value) * 100
        else:
            percentage_change = 0.0  

        
        plt.figure(figsize=(12, 6))
        
        plt.plot(timestamps, values, label=kpi_name, marker='o')
        plt.axhline(y=min_threshold, color='red', linestyle='--', label='Min Threshold')
        plt.axhline(y=max_threshold, color='green', linestyle='--', label='Max Threshold')
        plt.axhline(y=mean_value, color='blue', linestyle='--', label='Mean Value')

        
        plt.title(f'{kpi_name} Values Over Time (Alarmed at {alarm_time})\n'
                  f'Percentage Change from Mean: {percentage_change:.2f}%\n'
                  f'Mean: {mean_value:.2f}')
        plt.xlabel('Timestamp')
        plt.ylabel(kpi_name)
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()

        
        plt.show()

    def stream_data(self, data, alarm_handler=print):
        for _, row in data.iterrows():
            record = row.to_dict()
            alarms = self.process_record(record)
            for alarm in alarms:
                alarm_handler(alarm)

In [ ]:
import pandas as pd
import json
import yaml
import KPIOutlierMonitorJson
import os

def load_config_from_file(file_path):
        
    if file_path.endswith(".json"):
        with open(file_path, "r") as f:
            return json.load(f)
    elif file_path.endswith(".yaml") or file_path.endswith(".yml"):
        with open(file_path, "r") as f:
            return yaml.safe_load(f)
    else:
        raise ValueError("Unsupported file format. Please provide a .json or .yaml file.")
    
current_dir =  "C:\\Users\\2799361\\Desktop\\GenAI\\timeseries\\"

config_file_path = os.path.join(current_dir,'config.json')


config = load_config_from_file(config_file_path)

# print(config)
# print(config.items())
# print(config.keys())
# print(config.values())
# print(config['kpi_1'])
# print(config['kpi_1']['max'])


data = pd.read_csv(os.path.join(current_dir,"timeseries_kpis_with_outliers.csv"), parse_dates=["datetime"])


monitor = KPIOutlierMonitorJson.KPIOutlierMonitorJson(config)


def custom_alarm_handler(alarm):
    print(f"{alarm}")


monitor.stream_data(data, alarm_handler=custom_alarm_handler)